# Het enige verschil met #2 zijn de extra if statements rij 36 - 83 van cell 6
_Hier pas je automatisch delen van de text aan, zo maak je voor iedereen echt een eigen mail_

Om alle leden in een ketzer te mailen hebben we een aantal packages nodig. <br>
De meeste zitten standaard bij python, sommige niet: als je pandas of numpy niet hebt installeer die dan. <br>
Dat kan met: _pip install pandas_ en _pip install numpy_

In [2]:
# packages nodig voor mailen
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import time

# data management == heel handig
import pandas as pd
import numpy as np

We beginnen met het ophalen van ons excel, stop je excel in de zelfde map en voer de naam in waar staat _Leden van activiteit.xlsx_ . <br>
Zo lang het ketzer systeem het zelfde blijft hebben we altijd de kollomen 'Unnamed: 0', 'Uid', 'Lidnaam', 'Opmerking', 'Opmerking2', 'Lid sinds','Aangemeld door'. Deze behouden we maar gebruiken voor _'lidnaam'_ voor aanhef en _'Uid'_ voor mail.<br>
we slaan de eerste rij over, deze is altijd 'C.S.R. Delft - Beheer activiteiten', dat is de skiprows[0]

In [5]:
df_in = pd.read_excel("Leden op LustrumReis.xlsx")
df_in.columns

Index(['C.S.R. Lidnummer (niet studienummer) ', 'Voornaam (stek)',
       'Achternaam', 'Volledige naam', 'CiviNaam', 'Verticale', 'Mail',
       'Dinsdag Activiteit', 'Woensdag activiteit', 'Vrijdag Balton',
       'Busreis', 'Unnamed: 11', 'Allergieën ', 'Unnamed: 13', 'Kamer nummer',
       'Kamerindeling', 'Snurk je?', 'Ik vind gemengd man/vrouw slapen prima',
       'Op vakantie...', 'Meer mensen meer beter', 'Op mijn kamer zoek ik',
       'In de bus is mijn favoriete activiteit', 'Overige opmerkingen'],
      dtype='object')

Ter controlle de excel zie je dan hieronder:

In [6]:
df_in

,C.S.R. Lidnummer (niet studienummer),Voornaam (stek),Achternaam,Volledige naam,CiviNaam,Verticale,Mail,Dinsdag Activiteit,Woensdag activiteit,Vrijdag Balton,...,Unnamed: 13,Kamer nummer,Kamerindeling,Snurk je?,Ik vind gemengd man/vrouw slapen prima,Op vakantie...,Meer mensen meer beter,Op mijn kamer zoek ik,In de bus is mijn favoriete activiteit,Overige opmerkingen
0,x101,Jan,Lid,Jan Lid,Am. Lid,Primitus,janlidcsr@gmail.com,NaN,Szechenyi Baden,Nee,...,NaN,120,"Jan Lid, Jannette Lid",Beetje,Ja,...vind ik het fijn om wel nog een biertje in ...,Nee,Ik wil eigenlijk zo min mogelijk op mijn kamer...,-,Heel erg leuk ja
1,x102,Jannette,Lid,Jannette Lid,Am. Lid,Billy,jannettelidcsr@gmail.com,NaN,Szechenyi Baden,NaN,...,NaN,120,"Jan Lid, Jannette Lid",Beetje,Nee,...wil ik tot diep in de nacht borrelen om de ...,Nee,Ik wil eigenlijk zo min mogelijk op mijn kamer...,Hardlopen,Zin an! Doei.


Vervolgens laten we de lege kolom weg (unnamed:0) en kunnen we leden mailen met Uid@csrdelft.nl (ja vet handig)

In [14]:
# behouden de kollomen
df_in.rename(columns={"Kamer nummer":"Kamer_nummer",'C.S.R. Lidnummer (niet studienummer) ':"Uid"},inplace=True)
if "Uid" in df.columns: # check om error te voorkomen
    df = df.set_index("Uid")
    
# hier kan je nog een selectie maken indien wenselijk
df = df_in

# voegen een mail kollom aan
# dit geval bestaat ie al maar kan alsnog

# df['Mail'] = df['Uid'].apply(lambda x: f'{x}@csrdelft.nl')
df

,Voornaam (stek),Achternaam,Volledige naam,CiviNaam,Verticale,Mail,Dinsdag Activiteit,Woensdag activiteit,Vrijdag Balton,Busreis,...,Unnamed: 13,Kamer_nummer,Kamerindeling,Snurk je?,Ik vind gemengd man/vrouw slapen prima,Op vakantie...,Meer mensen meer beter,Op mijn kamer zoek ik,In de bus is mijn favoriete activiteit,Overige opmerkingen
Uid,,,,,,,,,,,,,,,,,,,,,
x101,Jan,Lid,Jan Lid,Am. Lid,Primitus,janlidcsr@gmail.com,NaN,Szechenyi Baden,Nee,Vliegtuig heen en terug,...,NaN,120,"Jan Lid, Jannette Lid",Beetje,Ja,...vind ik het fijn om wel nog een biertje in ...,Nee,Ik wil eigenlijk zo min mogelijk op mijn kamer...,-,Heel erg leuk ja
x102,Jannette,Lid,Jannette Lid,Am. Lid,Billy,jannettelidcsr@gmail.com,NaN,Szechenyi Baden,NaN,Ja,...,NaN,120,"Jan Lid, Jannette Lid",Beetje,Nee,...wil ik tot diep in de nacht borrelen om de ...,Nee,Ik wil eigenlijk zo min mogelijk op mijn kamer...,Hardlopen,Zin an! Doei.


Stel je wilt een deel los mailen, dan kan je makkelijkst selecteren op lid nummer.<br>
Bij echte uids kan je gwn een nummer ipv string invoeren<br>
Haal de twee # weg om te zien hoe het werkt (dit doe ik al hier boven)

Dan het echte verzenden:

In [17]:
############# Dingen om aan te passen #############
# deze is heel belangrijk, zorg dat hij bij testen op FALSE staat, anders verstuur je perongelijk mailtjes
send = False
# pas je commissie mail aan
sender_email = "commissie@csrdelft.nl"

############# Instellingen #############
# wat server instellingen belangrijk om verbinding te maken met de server, inprincipe generiek 
port = 587
smtp_server = "smtp.knorrie.org"

# prompt om je wachtwoord in te voergen: 
if send:    
    password = input("Type your password and press enter:\n")

# bijhouden voor handigheid
count_kamers = 0
# voor alle rijen in de excel
for i in range(len(df)):
    # per rij nemen we die specifieke
    row = df.iloc[i]
    
    ############# Deel wat anders is: #############
    # als de rij opmerkingen leeg is krijg je dus geen mail. 


    ############# Instellingen #############
    # maak een message opject aan 
    message = MIMEMultipart("alternative")

    # de verstuurder is de commissie
    message["From"] = sender_email

    # de ontavnger halen we uit de rij 
    receiver_email = row['Mail']
    message["To"] = receiver_email

    ############# Extra condities #############
    if str(row['Busreis']) == "Ja":
        bus = """We zien u zondag 26 juni om 13:30 bij de bushaltes op Delft centraal station."""
        
    elif str(row['Busreis']) == "Vliegtuig heen en terug":
        bus = """De bus vertrekt zondag 26 juni om 13:30 bij de bushaltes op Delft centraal station.
        U heeft aangegeven niet mee te gaan met de bus."""
        
    elif row['Busreis'] == 'Bus heen, maar niet terug':
        bus = """We zien u zondag 26 juni om 13:30 bij de bushaltes op Delft centraal station. 
        U heeft aangegeven op de terugweg niet mee te gaan met de bus."""
    
    elif row['Busreis'] == 'Bus heen, niet terug ':
        bus = """We zien u zondag 26 juni om 13:30 bij de bushaltes op Delft centraal station.
        U heeft aangegeven op de terugweg niet mee te gaan met de bus."""    
    else:
        raise IndexError(i,'Geen busreis data')
    
    kamer =  row.Kamerindeling.split(", ")
    verantwoordelijk = kamer[0]
        
    naam = str(row['Volledige naam'])

    try:
        kamer.remove(str(naam))

    except ValueError:
        try:
            if naam[-1] == " ":
                naam = naam[:-1]
            kamer.remove(str(naam))
            
        except ValueError:
            print(i,"removing failed",naam, kamer)
    
    for index, person in enumerate(kamer):
        if index == 0:
            str_kamer = person
        elif index == len(kamer) - 1:
            str_kamer += f" & {person}" 
        else:
            str_kamer += f", {person}" 
    
    if verantwoordelijk == row['Volledige naam']:
        count_kamers+=1
        sleutel = f"""Om de sleutel uitgifte bij aankomst bij het hostel te versnellen 
        bent <b>U</b> verantwoordelijk om die op te halen. Meer informatie volgt."""
    else:
        sleutel = f"""Om de sleutel uitgifte bij aankomst bij het hostel te versnellen is <b>{verantwoordelijk}</b>
        daar verantwoordelijk voor."""

    ############# Dingen om aan te passen #############
    #### gebruik bijv dit om de html opmaak te bouwen: https://html-online.com/editor/  of met de hand ####

    # geef de mail een title
    message["Subject"] = f"Voorbeeldmail naar alle leden in de activiteiten ketzer"
        # add main body
    # add main body
    body = f"""\
    <html>
      <body>
            <p>Beste {row.CiviNaam}, <br></p>
            <p>Nog 10 dagen te gaan! Hierbij wat laatste informatie:</p>
            
            <p>U slaapt op kamer {row.Kamer_nummer} met {str_kamer}.<br>
            In het hostel is beddengoed aanwezig, neem <b>wel</b> uw eigen <b>handoek</b> mee. 
            Eventueel zijn deze te huur bij het hostel.<br>
            {sleutel}</p>
            
            
            <p>{bus}<br>
            
            Er is beperkt plek voor bagage, dus houd het schappelijk. Denk aan een weekendtas/handbaggagekoffer.<br>
            
            We kunnen niet garanderen dat we een lange stop hebben bij een eetgelegenheid 
            dus neem zelf avondeten en ontbijt mee.
            </p>
            
            <p>Indien er iets niet duidelijk is horen wij het graag. 
            Houd de mededelingen 'wat-is-boven-applicatie' in de gaten.</p>

            <p>Met vriendelijke groet,
            <br> De 12e LustrumCie der C.S.R. Delft</p>
            
      </body>
    </html>

    """
    ############# Verzenden #############
    # je voegt dan de html toe
    message.attach(MIMEText(body, "html"))

    # je kan ook een foto toevoegen bijv. 
    picture = False
    if picture:
        with open('Postergala.jpeg','rb') as fin:
            img = MIMEImage(fin.read())
            img.add_header('Content-Disposition', 'inline; filename=poster.jpg')
            message.attach(img)


    # Als je aan het testen ben kan je printen aan zetten en dan zie je een voorbeeld
    printing = True

    # dit verstuurd het daadwerkelijk
    if send:
        context = ssl.create_default_context()
        with smtplib.SMTP(smtp_server, port) as server:
            server.ehlo()  # Can be omitted
            server.starttls(context=context)
            server.ehlo()  # Can be omitted
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, message.as_string())

        print(f"{i}: {receiver_email} sent")

    # als het een test is print hij het het bericht
    else:
        if printing:
            print(message)

    # als je deze er niet in doet stopt ie na 100 mails
    time.sleep(1)
print(f'{count_kamers} kamers')

Content-Type: multipart/alternative; boundary="===============7475205501755920331=="
MIME-Version: 1.0
From: commissie@csrdelft.nl
To: janlidcsr@gmail.com
Subject: Voorbeeldmail naar alle leden in de activiteiten ketzer

--===============7475205501755920331==
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

    <html>
      <body>
            <p>Beste Am. Lid, <br></p>
            <p>Nog 10 dagen te gaan! Hierbij wat laatste informatie:</p>
            
            <p>U slaapt op kamer 120 met Jannette Lid.<br>
            In het hostel is beddengoed aanwezig, neem <b>wel</b> uw eigen <b>handoek</b> mee. 
            Eventueel zijn deze te huur bij het hostel.<br>
            Om de sleutel uitgifte bij aankomst bij het hostel te versnellen 
        bent <b>U</b> verantwoordelijk om die op te halen. Meer informatie volgt.</p>
            
            
            <p>De bus vertrekt zondag 26 juni om 13:30 bij de bushaltes op Delft centraal s

Hierboven een voorbeeld mail in tekst en dit heironder hoe het in gmail er uit ziet

![Hierboven een voorbeeld mail ](Voorbeeld2.png)